### Machine Learning Example Notebook



### Imports 

In [1]:
import pandas as pd
import numpy as np

### Data preparation

* Read input data from csv files
* Calculate fingerprints
* Get labels

In [2]:
actives_df = pd.read_csv("../data/Chembl_EGFR_actives_ML.csv")
actives_df.head()

,Unnamed: 0,chembl_id,canonical_smiles
0,5792,CHEMBL3946232,Fc1ccc(Nc2ncnc3cc(OCCCN4CCOCC4)c(NC(=O)C5=CCCC...
1,2318,CHEMBL443171,OC(=O)C1=C(C(=O)c2c(O)cc(O)cc2O1)c3ccc(O)cc3
2,911,CHEMBL163188,CN(C)c1cc2c(Nc3cccc(C)c3)ncnc2cn1
3,997,CHEMBL286343,Brc1cccc(Nc2ncnc3cc(OCCCn4ccnc4)c(NC(=O)C=C)cc...
4,2566,CHEMBL514436,CS(=O)(=O)CCNCCCCOc1ccc2ncnc(Nc3ccc(OCc4cccc(F...


In [3]:
actives_df.drop('Unnamed: 0', axis=1, inplace=True)
actives_df.head()

,chembl_id,canonical_smiles
0,CHEMBL3946232,Fc1ccc(Nc2ncnc3cc(OCCCN4CCOCC4)c(NC(=O)C5=CCCC...
1,CHEMBL443171,OC(=O)C1=C(C(=O)c2c(O)cc(O)cc2O1)c3ccc(O)cc3
2,CHEMBL163188,CN(C)c1cc2c(Nc3cccc(C)c3)ncnc2cn1
3,CHEMBL286343,Brc1cccc(Nc2ncnc3cc(OCCCn4ccnc4)c(NC(=O)C=C)cc...
4,CHEMBL514436,CS(=O)(=O)CCNCCCCOc1ccc2ncnc(Nc3ccc(OCc4cccc(F...


In [4]:
inactives_df = pd.read_csv("../data/Chembl_EGFR_inactives_ML.csv")
inactives_df.drop('Unnamed: 0', axis=1, inplace=True)
inactives_df.head()

,chembl_id,canonical_smiles
0,CHEMBL4066684,CC1=CC(=O)N(N=C1C(=O)Nc2ccc(Oc3ccnc4[nH]ccc34)...
1,CHEMBL344652,CN(C)\N=N\c1ccc2ncnc(N(C)c3ccccc3)c2c1
2,CHEMBL1929306,CCOc1ccccc1c2oc(SCc3ccccc3)nn2
3,CHEMBL144760,Oc1ccc(\C=C(/C#N)\C(=O)NCCCCCCNC(=O)\C(=C\c2cc...
4,CHEMBL1928886,Oc1ccc(CNc2ccc3ncnc(Nc4cccc(Cl)c4)c3c2)cc1


*Count number of actives and inactives*

In [5]:
print(f"Actives: {len(actives_df)} \t Inactives: {len(inactives_df)}")

Actives: 106 	 Inactives: 105


##### Collect molecules' smiles in array


In [6]:
smiles_active = actives_df["canonical_smiles"].tolist()
smiles_inactive = inactives_df["canonical_smiles"].tolist()

##### Calculate molecular fingerprints

In [7]:
from rdkit import Chem
from rdkit.Chem import AllChem

fingerprints_active = []
fingerprints_inactive = []

for smiles in smiles_active: 
    molecule = Chem.MolFromSmiles(smiles)
    fingerprints_active.append(AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=1024))

for smiles in smiles_inactive:
    molecule = Chem.MolFromSmiles(smiles)
    fingerprints_inactive.append(AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=1024))

Concatenate fingerprint arrays

In [8]:
fps = fingerprints_active + fingerprints_inactive

##### Prepare classifier assignment
* active = 1
* inactive = 0

In [9]:
# 'Active' = 1
y_active = np.ones(len(fingerprints_active))
y_active

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [10]:
# 'Inactive' = 0
y_inactive = np.zeros(len(fingerprints_inactive))
y_inactive

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

Concatenate label arrays

In [11]:
y = np.concatenate([y_active, y_inactive])

### Random forest (RF)

* Split data in test and training set
* Train the RF model
* Test the performance

##### Split data in train and test set

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(fps, y, test_size=0.20)

##### Train the RF model
See http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html for an explanation of the parameter.

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [15]:
forest = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_features=100)

Fit the data (Learn the pattern)

In [16]:
forest.fit(X_train, y_train)

RandomForestClassifier(max_features=100, n_jobs=-1)

##### Test performance of the model

In [17]:
from sklearn.metrics import recall_score

Predict the class (active/inactive) for the test data (X_test)

In [18]:
y_pred = forest.predict(X_test) 
y_pred[:10]

array([1., 1., 1., 0., 1., 1., 0., 0., 1., 1.])

Predict probabilites for the class annotation

In [19]:
y_prob = forest.predict_proba(X_test)
y_prob[:10]

array([[0.39, 0.61],
       [0.03, 0.97],
       [0.07, 0.93],
       [0.84, 0.16],
       [0.2 , 0.8 ],
       [0.03, 0.97],
       [0.79, 0.21],
       [0.6 , 0.4 ],
       [0.05, 0.95],
       [0.23, 0.77]])

Calculate the model accuracy on this test set

In [20]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.2f}")

Model accuracy: 0.79


Calculate the area under the ROC durce (AUC)

In [21]:
roc_auc = metrics.roc_auc_score(y_test, y_prob[:,1])  
print(f"Model AUC: {roc_auc:.2f}")

Model AUC: 0.88


### Make predictions on a new data set
Here: Test on FDA approved drugs

In [22]:
egfr_compounds = pd.read_csv("../data/EGFR.smi", names=["smiles", "names"], index_col=1)
egfr_compounds.head()

,smiles
names,
Gefitinib,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1
Erlotinib,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1
Lapatinib,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...
Afatinib,CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...
Osimertinib,C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(O...


In [23]:
print("Name\t\t\tPrediction\tProbability")
print("----\t\t\t----------\t-----------")
for name, row in egfr_compounds.iterrows():
    molecule = Chem.MolFromSmiles(row["smiles"])
    egfr_fingerprints = AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=1024)
    # shape of input has to be adapted 
    egfr_fingerprints =  np.reshape(egfr_fingerprints, (1, -1))
    y_pred = forest.predict(egfr_fingerprints)
    y_prob = forest.predict_proba(egfr_fingerprints)
    print(name, y_pred, y_prob, sep="\t\t")

Name			Prediction	Probability
----			----------	-----------
Gefitinib		[1.]		[[0.25 0.75]]
Erlotinib		[1.]		[[0.03 0.97]]
Lapatinib		[1.]		[[0.07 0.93]]
Afatinib		[1.]		[[0.05 0.95]]
Osimertinib		[0.]		[[0.62 0.38]]
